In [13]:
%reset -f

import pdfkit # apt-get install -y wkhtmltopdf
import pandas as pd
from google.colab import files
import unicodedata
from datetime import datetime, date
import os
import shutil
import zipfile


In [15]:
def formatar_string(string: str) -> str:
  """
  função que formata uma string e retorna ela
    - remove caracteres de controle como \n
    - remove letras com acento e deixa ela normal à -> a
    - deixa tudo maiusculo
  """

  if not isinstance(string, str):
    return string

  string = (
    unicodedata.normalize("NFKD", string) # separa acentos (ex: "ç" → "c"+"¸")
    .encode("ascii", "ignore")            # remove tudo que não for ascii
    .decode("utf8")                       # converte de volta para string
    .upper()                              # tudo maisúculo
    .strip()                              # remove espaços
    )

  for char in ["\xa0", "\n", "\r", ":"]: # retira alguns caracteres especiais
    string = string.replace(char, "")

  return string


def formatar_df_geral(df: pd.DataFrame) -> None:
  """
  função que recebe uma df e modifica ela da seguinte forma:
    - formata as colunas com o "formatar_string()"
    - remove colunas do forms (como email, data de envio, etc)
    - formata a data para o formato datetime.date
    - mantem apenas a letra do turno (A, B ou C)
    - remove as duplicatas (considera a frota caso exista na df)
  """

  df.columns = [formatar_string(coluna) for coluna in df.columns] # formata os nomes das colunas

  colunas_extra = ["HORA DE INICIO", "HORA DE CONCLUSAO", "EMAIL", "NOME", "HORA DA ULTIMA MODIFICACAO"]
  df.drop(columns=colunas_extra, errors='ignore', inplace=True) # remove algumas colunas que são do forms

  df['DATA'] = pd.to_datetime(df['DATA'], errors='coerce').dt.date # converte em datetime.date
  df["TURNO"] = df["TURNO"].str[-1:] # deixa apenas a letra (ultimo char)

  remove = ['TURNO','DATA'] # colunas que serão usadas para identificar as duplicatas
  if 'FROTA' in df.columns: # caso a df contenha 'FROTA', consideramos ela na lógica de remoção
    remove.append('FROTA')

  df.sort_values(by=['TURNO', 'ID'], ascending=[True, False], inplace=True) # ordena por turno e id
  df.drop_duplicates(subset=remove, keep='first', inplace=True) # vai manter só o primeiro (mais recente)


def formatar_nome(nome: str) -> str:
  """
  deixa o nome do arquivo certinho, apenas com checklist + nome
    - recebe uma string
    - remove o . do final e tudo que estiver depois dele
    - remove os parenteses com o numero de respostas
    - passa pelo "formatar_string()"
    - retorna a string
  """

  if "." in nome:
    nome = nome.split(".", 1)[0] # guarda só o que fica antes do ponto

  while "(" in nome and ")" in nome: # retira o que estiver nos parenteses
    inicio = nome.find("(")
    fim = nome.find(")", inicio)
    if fim == -1:
        break
    nome = nome[:inicio] + nome[fim+1:]

  return formatar_string(nome) # passa pelo formatador


def cortar_nome(nome: str) -> str:
  """
  remove a parte do "CHECKLIST" ou "CHECK LIST " e retorna só o primeiro nome
  """

  return nome.replace("CHECK LIST ", "").replace("CHECKLIST ", "").split(" ")[0]


def classificar_colunas(df: pd.DataFrame) -> dict:
  """
  classifica as colunas de uma df em três tipos e retorna um dicionario com {coluna : tipo}
    - opcao_abc: utilizada na coluna TURNO
    - opcao_sim_nao: quando a resposta é de sim ou não (bolinhas)
    - resposta: quando a resposta é de texto
  """

  tipos = {}

  for col in df.columns:
    valores = df[col].dropna().astype(str).map(formatar_string)
    valores_unicos = set(valores.unique())

    if col == "TURNO":
      tipos[col] = "opcao_abc"
      continue

    if valores_unicos.issubset( {"SIM", "NAO"}) and len(valores_unicos) <= 2:
      tipos[col] = "opcao_sim_nao"
      continue

    if valores_unicos.issubset({"BOM", "MEDIO", "RUIM"}) and len(valores_unicos) <= 3:
      tipos[col] = "opcao_conservacao"
      continue

    tipos[col] = "resposta"

  return tipos


def criar_dict_checklists(arquivos: dict) -> dict:
  """
  recebe os arquivos do excel em forma de dicionário para gerar as dfs
    - a chave sera o primeiro nome da checklist
    - o nome será o nome completo formatado
    - o df será o dataframe de respostas
    - o culunas sera um dicionário com o nome da coluna e seu tipo {coluna : tipo}

    - formata as colunas e remove as denecessárias
    - remove entradas duplicadas, mantendo apenas a primeira
    - separa o caminhão e a empilhadeira pela frota
  """

  checklists = {}

  for arquivo in arquivos.keys():
    titulo_checklist = formatar_nome(arquivo) # nome completo como CHECK LIST PA
    chave_checklist = cortar_nome(titulo_checklist) # a chave do checklist, apenas PA
    df = pd.read_excel(arquivo)
    formatar_df_geral(df) # dataframe do checklist com todas as respostas
    colunas = classificar_colunas(df) # classifica as colunas com seu tipo

    if chave_checklist == 'CAMINHAO':
      df['FROTA'] = df['FROTA'].astype('string')
      checklists[chave_checklist + " 298"] = {"titulo": titulo_checklist + " 298", "df": df[df['FROTA'].str.contains('298')].copy(), "colunas": colunas}
      checklists[chave_checklist + " 302"] = {"titulo": titulo_checklist + " 302", "df": df[df['FROTA'].str.contains('302')].copy(), "colunas": colunas}
      continue

    if chave_checklist == 'EMPILHADEIRA':
      df['FROTA'] = df['FROTA'].astype('string')
      checklists[chave_checklist + " 120"] = {"titulo": titulo_checklist + " 120", "df": df[df['FROTA'].str.contains('120')].copy(), "colunas": colunas}
      checklists[chave_checklist + " 155"] = {"titulo": titulo_checklist + " 155", "df": df[df['FROTA'].str.contains('155')].copy(), "colunas": colunas}
      continue

    checklists[chave_checklist] = {"titulo": titulo_checklist, "df": df, "colunas": colunas}

  return checklists


uploaded = files.upload()
checklists = criar_dict_checklists(uploaded)

for checklist in checklists.keys():
    print(checklist)


Saving CHECK LIST CAMINHÃO BASCULANTE ARTICULADO(1-1296).xlsx to CHECK LIST CAMINHÃO BASCULANTE ARTICULADO(1-1296) (1).xlsx
Saving CHECK LIST PÁ CARREGADEIRA(1-651).xlsx to CHECK LIST PÁ CARREGADEIRA(1-651) (1).xlsx
Saving CHECK LIST EMPILHADEIRA(1-1495).xlsx to CHECK LIST EMPILHADEIRA(1-1495) (1).xlsx
Saving CHECK LIST ESTEIRA(1-747).xlsx to CHECK LIST ESTEIRA(1-747) (1).xlsx
Saving CHECK LIST FERRAMENTAS(1-128).xlsx to CHECK LIST FERRAMENTAS(1-128) (1).xlsx
Saving CHECK LIST BALANÇA TOLFLUX(1-755) (1).xlsx to CHECK LIST BALANÇA TOLFLUX(1-755) (1) (1).xlsx
CAMINHAO 298
CAMINHAO 302
PA
EMPILHADEIRA 120
EMPILHADEIRA 155
ESTEIRA
FERRAMENTAS
BALANCA


In [29]:
def gerar_periodo(inicio, fim) -> pd.DatetimeIndex:
  """
  retorna um periodo do pandas com base em uma data de inicio e uma de fim
  """

  data_inicio = pd.to_datetime(inicio).date()
  data_fim = pd.to_datetime(fim).date()
  return pd.date_range(start=data_inicio, end=data_fim).date

periodo = gerar_periodo("2025-11-30", "2025-12-09")
turnos = ['C']


In [30]:
def gerar_pendencias(checklists: dict, periodo: pd.DatetimeIndex, turnos: list) -> pd.DataFrame:
  df_pendencias= pd.DataFrame(
    [(data, t) for data in periodo for t in turnos],
    columns=["DATA", "TURNO"]
  )

  for checklist in checklists.keys():
    checklists[checklist]['df']["DATA"] = pd.to_datetime(
        checklists[checklist]['df']["DATA"]
    ).dt.date

    df_periodo = checklists[checklist]['df'][
        checklists[checklist]['df']["DATA"].isin(periodo)
    ].copy()

    df_periodo["TURNO"] = (
        df_periodo["TURNO"]
        .astype(str)
        .str.upper()
        .str.strip()
        .str[-1:]
    )

    chaves_reais = set(zip(df_periodo["DATA"], df_periodo["TURNO"]))

    df_pendencias[checklist] = [
        "PENDENTE" if (d, t) not in chaves_reais else None
        for d, t in zip(df_pendencias["DATA"], df_pendencias["TURNO"])
    ]

  pendencias = df_pendencias[
    df_pendencias.drop(columns=['DATA', 'TURNO']).notnull().any(axis=1)
  ].replace({None:""})

  return pendencias


gerar_pendencias(checklists, periodo, turnos)

,DATA,TURNO,CAMINHAO 298,CAMINHAO 302,PA,EMPILHADEIRA 120,EMPILHADEIRA 155,ESTEIRA,FERRAMENTAS,BALANCA
1,2025-12-01,C,,PENDENTE,,,,,,
3,2025-12-03,C,,,PENDENTE,,,,,PENDENTE
5,2025-12-05,C,,,PENDENTE,PENDENTE,PENDENTE,PENDENTE,PENDENTE,PENDENTE
6,2025-12-06,C,,,PENDENTE,,,PENDENTE,PENDENTE,PENDENTE
7,2025-12-07,C,,,,,,,PENDENTE,PENDENTE
8,2025-12-08,C,,,,,,,PENDENTE,PENDENTE
9,2025-12-09,C,,,,,,PENDENTE,PENDENTE,PENDENTE


In [ ]:
def gerar_relatorio(checklist: dict) -> str:
    """
    gera um html com base no dict gerado pelo "criar_dict_checklists()"
    """

    html = f"""
    <!DOCTYPE html>
    <html>
    <head>
    <meta charset="UTF-8">
    <style>
    body {{ font-family: Arial, sans-serif; padding: 20px; line-height: 1.5; }}
    h1 {{ color: #2F4F4F }}
    .questao {{ display: flex; flex-direction: column; margin: 10px 0px; page-break-inside: avoid; break-inside: avoid; }}
    .enunciado {{ font-weight: bold; margin-top: 5px; }}
    .opcoes {{ margin-left: 5px; display: flex; margin-right: 12px; }}
    .opcoes > div {{ display: flex; align-items: center; margin-right: 12px; }}
    .bolinha {{ margin-right: 5px; width: 15px; height: 15px; border-radius: 50%; display: inline-block; border: 2px solid currentColor; background-color: transparent; }}
    .opcao-sim .sim {{ background-color: black; }}
    .opcao-nao .nao {{ background-color: black; }}
    .opcao-a .a {{ background-color: black; }}
    .opcao-b .b {{ background-color: black; }}
    .opcao-c .c {{ background-color: black; }}
    .opcao-bom .bom {{ background-color: black; }}
    .opcao-medio .medio {{ background-color: black; }}
    .opcao-ruim .ruim {{ background-color: black; }}
    .nao-respondida, .nao-respondida .bolinha {{ color: gray; background-color: transparent; }}
    .resposta {{ margin-left: 5px; }}
    </style>
    </head>

    <body>
    <header>
      <h1>{checklist['titulo']}</h1>
    </header>
    """

    for pergunta, tipo in checklist['colunas'].items():
        resposta = checklist['df'][pergunta]
        nao_respondida = pd.isna(resposta) or str(resposta).strip() == ""

        if nao_respondida:
            html += f"""
            <div class="questao nao-respondida">
              <span class="enunciado">{pergunta}</span>
            """
            if tipo == "resposta":
              html += "<span class=\"resposta\">! - Sem resposta.</span>"
            if tipo == "opcao_sim_nao":
                html += """
                <div class="opcoes">
                <div><span class="bolinha sim"></span>Sim</div>
                <div><span class="bolinha nao"></span>Não</div></div>
                """
            elif tipo == "opcao_abc":
                html += """
                <div class="opcoes">
                <div><span class="bolinha a"></span>A</div>
                <div><span class="bolinha b"></span>B</div>
                <div><span class="bolinha c"></span>C</div></div>
                """
            html += "</div>"
            continue

        if tipo == "resposta":
            if pergunta == 'DATA':
              resposta = resposta.strftime("%d-%m-%Y")

            html += f"""
            <div class="questao">
              <span class="enunciado">{pergunta}</span>
              <span class="resposta">{resposta}</span>
            </div>
            """

        elif tipo == "opcao_sim_nao":
            resposta = "opcao-sim" if str(resposta).strip().upper() == "SIM" else "opcao-nao"

            html += f"""
            <div class="questao">
              <span class="enunciado">{pergunta}</span>
              <div class="opcoes {resposta}">
                <div><span class="bolinha sim"></span>Sim</div>
                <div><span class="bolinha nao"></span>Não</div>
              </div>
            </div>
            """
        elif tipo == "opcao_conservacao":
          r = str(resposta).strip().upper()
          classe = "opcao-bom" if r == "BOM" else ("opcao-medio" if r == "MEDIO" else "opcao-ruim")

          html += f"""
          <div class="questao">
            <span class="enunciado">{pergunta}</span>
            <div class="opcoes {classe}">
              <div><span class="bolinha bom"></span>Bom</div>
              <div><span class="bolinha medio"></span>Médio</div>
              <div><span class="bolinha ruim"></span>Ruim</div>
            </div>
          </div>
          """

        elif tipo == "opcao_abc":
            resposta = f"opcao-{str(resposta).strip().lower()[-1]}"

            html += f"""
            <div class="questao">
              <span class="enunciado">{pergunta}:</span>
              <div class="opcoes {resposta}">
                <div><span class="bolinha a"></span>A</div>
                <div><span class="bolinha b"></span>B</div>
                <div><span class="bolinha c"></span>C</div>
              </div>
            </div>
            """

    html += "</body></html>" # fecha as tags de body e html (pdfkit as vezes buga)

    return html


def gerar_nome_pdf(checklist: pd.DataFrame) -> str:
  """
  gera o nome correto para nomear o arquivo pdf
  - caso seja uma checklist de pa e tenha frota, coloca no nome
  - se não, apenas o turno + nome do checklist + data
  """

  turno = f"TURNO {checklist['df']['TURNO']}"
  nome = checklist['titulo']
  data = checklist['df']['DATA'].strftime("%d-%m-%Y")

  if 'PA' in nome:
    frota = checklist['df'].get('FROTA', '')
    frota_str = str(frota).strip()
    if not pd.isna(frota) and frota_str != "":
        return f"{turno} - {nome} {frota_str} - {data}.pdf"

  return f"{turno} - {nome} - {data}.pdf"


def imprimir_respostas(checklists: dict, periodo: pd.DatetimeIndex, turnos: list) -> None:
  """
  gera arquivos pdf para cada resposta do formulario dentro daquele periodo
  coloca em pastas, separado pelo turno e pela checklist
  """

  dir_raiz = "/content/pdfs"

  if os.path.exists(dir_raiz):
    shutil.rmtree(dir_raiz)

  os.makedirs(dir_raiz)

  for nome, checklist in checklists.items():
    df = checklist['df']

    for turno in turnos:
      pasta_checklist = f"{dir_raiz}/{turno}/{nome}"
      os.makedirs(pasta_checklist)

      for data in periodo:
        resposta = df[(df["DATA"] == data) & (df["TURNO"] == turno)]

        if resposta.empty:
          continue

        linha = resposta.iloc[0]

        checklist_tmp = {
            "titulo": checklist["titulo"],
            "colunas": checklist["colunas"],
            "df": linha
        }

        html = gerar_relatorio(checklist_tmp)
        nome_pdf = gerar_nome_pdf(checklist_tmp)

        caminho_pdf = f"{pasta_checklist}/{nome_pdf}"

        pdfkit.from_string(html, caminho_pdf, {
            "encoding": "UTF-8",
            "dpi": 150,
            "page-size": "A4"
        })

  for turno in turnos:
    zip_path = f"{dir_raiz}/TURNO_{turno}.zip"
    pasta_turno = f"{dir_raiz}/{turno}"

    with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zipf:
      for raiz, dirs, arquivos in os.walk(pasta_turno):
        for arquivo in arquivos:
          caminho_completo = os.path.join(raiz, arquivo)
          caminho_relativo = os.path.relpath(caminho_completo, pasta_turno)
          zipf.write(caminho_completo, caminho_relativo)

imprimir_respostas(checklists, periodo, turnos)
